In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)

respone = llm.invoke("What is the capital of France?")
print(respone.content)

The capital of France is Paris.


In [2]:
import chromadb

client = chromadb.Client()
collection = client.create_collection("test")


In [3]:
collection.add(
    documents=["This doc is about India", "This doc is about France"],
    ids=["id1", "id2"],
)

In [4]:
docs  = collection.get()
print(docs)

{'ids': ['id1', 'id2'], 'embeddings': None, 'documents': ['This doc is about India', 'This doc is about France'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None, None]}


In [ ]:
collection.query(
    query_texts=["Which cuisine does biryani belong to?"],
    n_results=2,
    
)

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['This doc is about India', 'This doc is about France']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.6569032669067383, 1.8689751625061035]]}

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-i-itc/job/R-51569")
data = loader.load().pop().page_content
print(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Software Engineer I , ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu





In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED DATA FROM WEBSITE:
    ({data})
    ### INSTRUCTION:
    The scraped texy is from the career's page of a website.
    YOur job is to extract the job postings and return them in a JSON format containing the following keys: 'role','experience','skills' and description.
    Only return the valid JSON without any preamble or explanation and no need to mention it as json in triple quotes.
    ### VALID JSON(NO PREAMBLE):
    """
)

extract = prompt_extract | llm
res = extract.invoke(input={"data": data})
print(res.content)

{
    "role": "Software Engineer I, ITC",
    "experience": "1+ years of professional experience as a Full Stack Software Engineer or in a similar role",
    "skills": [
        "Front-End Skills in React or similar technologies",
        "Server-side programming languages such as Node.js, Python or similar",
        "NoSQL databases",
        "API Development",
        "Cloud Technologies (AWS)",
        "Version Control (Git, GitHub)",
        "Testing Frameworks (unit testing, integration testing, TDD)"
    ],
    "description": "Design & Develop scalable web applications and services, Collaborate with product managers, designers, and other engineers, Implement dynamic and user-friendly web interfaces, Design and implement RESTful APIs, microservices, and backend services"
}


In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_str = json_parser.parse(res.content)
print(json_str)

{'role': 'Software Engineer I, ITC', 'experience': '1+ years of professional experience as a Full Stack Software Engineer or in a similar role', 'skills': ['Front-End Skills in React or similar technologies', 'Server-side programming languages such as Node.js, Python or similar', 'NoSQL databases', 'API Development', 'Cloud Technologies (AWS)', 'Version Control (Git, GitHub)', 'Testing Frameworks (unit testing, integration testing, TDD)'], 'description': 'Design & Develop scalable web applications and services, Collaborate with product managers, designers, and other engineers, Implement dynamic and user-friendly web interfaces, Design and implement RESTful APIs, microservices, and backend services'}


In [9]:
type(json_str)

dict

In [10]:
import pandas as pd
 
df = pd.read_csv("my_portfolio.csv")

df.head()


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [11]:
import chromadb
import uuid

client =  chromadb.PersistentClient('vectorstore')

collection = client.get_or_create_collection(name="my_portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas={"links": row["Links"]},
            ids = [str(uuid.uuid4())])


In [15]:
links = collection.query(query_texts=["Experience in Python","Experience in Java"], n_results=2).get("metadatas",[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [12]:
job = json_str
job['skills']

['Front-End Skills in React or similar technologies',
 'Server-side programming languages such as Node.js, Python or similar',
 'NoSQL databases',
 'API Development',
 'Cloud Technologies (AWS)',
 'Version Control (Git, GitHub)',
 'Testing Frameworks (unit testing, integration testing, TDD)']

In [16]:
job

{'role': 'Software Engineer I, ITC',
 'experience': '1+ years of professional experience as a Full Stack Software Engineer or in a similar role',
 'skills': ['Front-End Skills in React or similar technologies',
  'Server-side programming languages such as Node.js, Python or similar',
  'NoSQL databases',
  'API Development',
  'Cloud Technologies (AWS)',
  'Version Control (Git, GitHub)',
  'Testing Frameworks (unit testing, integration testing, TDD)'],
 'description': 'Design & Develop scalable web applications and services, Collaborate with product managers, designers, and other engineers, Implement dynamic and user-friendly web interfaces, Design and implement RESTful APIs, microservices, and backend services'}

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Software Engineering Services for Scalable Web Applications

Dear Hiring Manager,

I came across your job posting for a Software Engineer I, ITC, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can fulfill your requirements. With over 1+ years of experience in professional software engineering, our team is well-equipped to design and develop scalable web applications and services.

Our expertise aligns with your requirements, including Front-End Skills in React, server-side programming languages such as Node.js and Python, NoSQL databases, API Development, Cloud Technologies (AWS), Version Control (Git, GitHub), and Testing Frameworks (unit testing, integration testing, TDD). We have a proven track record of collaborating with product managers, designers, and other engineers to deliver dynamic and user-friendly web interfaces, RESTful APIs, microservices, and backend services.

At AtliQ, we have empowered numerous enterprises with tailored s

: 